# UDS emulator 

## Preliminaries

In [1]:
from my_imports import *

The current branch is: Badly_Implemented_Temporal_Bundling


In [2]:
yaml_path = 'config_file.yaml'
yaml_data = utils.load_yaml(yaml_path)

training_simulations_path       = yaml_data['training_simulations_path']
validation_simulations_path     = yaml_data['validation_simulations_path']

inp_path                        = yaml_data['inp_path']

training_windows_path           = yaml_data['training_windows_path']
use_saved_training_windows      = yaml_data['use_saved_training_windows']

validation_windows_path         = yaml_data['validation_windows_path']
use_saved_validation_windows    = yaml_data['use_saved_validation_windows']

num_events_training             = yaml_data['num_events_training']
num_events_validation           = yaml_data['num_events_validation']

steps_behind                    = yaml_data['steps_behind']
steps_ahead                     = yaml_data['steps_ahead']
epochs                          = yaml_data['epochs']



## Create simulation objects from data

In [3]:
list_of_training_simulations    = os.listdir(training_simulations_path)
list_of_validation_simulations  = os.listdir(validation_simulations_path)

random.shuffle(list_of_training_simulations)

print('Number of training simulations: ', len(list_of_training_simulations))
print('Number of validation simulations: ', len(list_of_validation_simulations))

Number of training simulations:  161
Number of validation simulations:  29


In [4]:
training_simulations = utils.extract_simulations_from_folders(training_simulations_path, inp_path, num_events_training)

The file does not have [STORAGE]

The file does not have '[PUMPS]\n'
The file does not have '[ORIFICES]\n'
The file does not have '[WEIRS]\n'


In [5]:
if use_saved_training_windows:
    training_windows = utils.load_windows(training_windows_path)
else:
    training_windows = utils.get_all_windows_from_list_simulations(training_simulations, steps_ahead, steps_behind)

In [6]:
# utils.save_pickle(training_windows, 'saved_objects/training_windows_debugging.pk')

In [7]:
validation_simulations = utils.extract_simulations_from_folders(validation_simulations_path, inp_path, num_events_validation)

The file does not have [STORAGE]

The file does not have '[PUMPS]\n'
The file does not have '[ORIFICES]\n'
The file does not have '[WEIRS]\n'


In [8]:
if use_saved_validation_windows:
    validation_windows = utils.load_windows(validation_windows_path)
else:
    validation_windows = utils.get_all_windows_from_list_simulations(validation_simulations, steps_ahead = 60, steps_behind = steps_behind)

In [9]:
print('Number of training windows: ',   len(training_windows))
print('Number of validation windows: ', len(validation_windows))

Number of training windows:  1676
Number of validation windows:  37


## Normalizer

In [10]:
normalizer = Normalizer(training_windows)

In [11]:
normalized_training_windows = normalizer.get_list_normalized_training_windows()
training_loader = DataLoader(normalized_training_windows, batch_size=32, shuffle = True)

In [12]:
normalized_validation_windows = [normalizer.normalize_window(val_win) for val_win in validation_windows]
validation_loader = DataLoader(normalized_validation_windows, batch_size=32)

## Training GNN

In [15]:
gnn_model = GNNModel(steps_ahead=1, steps_behind=6)
# gnn_model.load_state_dict(torch.load('saved_objects/model_trained_4steps.pt'))

optimizer = optim.Adam(gnn_model.parameters(), lr = 0.01)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
loss_fn   = nn.MSELoss()

In [16]:
history = train(
    model       = gnn_model,
    optimizer   = optimizer,
    scheduler   = scheduler,
    loss_fn     = loss_fn,
    train_dl    = training_loader,
    val_dl      = validation_loader,
    epochs      = epochs,
    device      = 'cpu',
    report_freq = 2
    )

train() called:model=GNNModel, opt=Adam(lr=0.010000), epochs=20,device=cpu

Best model so far at epoch:  1
Epoch   1/ 20, train loss:  4.42, val loss:  2.09
Epoch   2/ 20, train loss:  0.46, val loss:  2.30
Best model so far at epoch:  3
Epoch   4/ 20, train loss:  0.29, val loss:  1.64
Epoch   6/ 20, train loss:  0.19, val loss:  1.58
Epoch   8/ 20, train loss:  0.18, val loss:  1.00
Epoch  10/ 20, train loss:  0.17, val loss:  1.02
Best model so far at epoch:  11
Epoch  12/ 20, train loss:  0.15, val loss:  1.18
Epoch  14/ 20, train loss:  0.14, val loss:  3.26
Epoch  16/ 20, train loss:  0.14, val loss:  1.64
Epoch  18/ 20, train loss:  0.13, val loss:  0.87
Best model so far at epoch:  20
Epoch  20/ 20, train loss:  0.13, val loss:  0.60
Best model found at epoch:  20
This best model has this validation loss:  0.5976412695285084

Time total:     647.73 sec
Time per epoch: 32.39 sec


In [ ]:
# torch.save(gnn_model.state_dict(), 'saved_objects/trained_model_{}_epochs_{}_steps.pt'.format(epochs, steps_ahead))

In [17]:
vis.plot_loss(history)

## Using the GNN

In [18]:
val_event =  validation_simulations[-1]
print(val_event)

runoff_pd           = val_event.runoff_raw_data

sim_in_window       = val_event.get_simulation_in_one_window(steps_behind = 1)

norm_sim_in_window  = normalizer.normalize_window(sim_in_window)

swmm_heads_pd       = normalizer.get_unnormalized_heads_pd(sim_in_window['y']) 
predicted_heads_pd  = normalizer.get_unnormalized_heads_pd(gnn_model(norm_sim_in_window))

SWMMSimulation(real_90)


In [ ]:
importlib.reload(vis)

In [33]:
node =  'j_90369'
vis.plot_heads_timeseries(swmm_heads_pd, predicted_heads_pd, runoff_pd, node)

## Visualization of results

In [ ]:
importlib.reload(vis)

In [ ]:
error = predicted_heads_pd - swmm_heads_pd

In [ ]:
error= error.transpose().to_numpy()

In [ ]:
error = abs(error)
fig = vis.plot_nodal_variable(error, norm_sim_in_window, 'error in head (m) ', colorscale = 'OrRd')


In [ ]:
norm_sim_in_window.name_nodes[0]

In [ ]:
# fig.write_html('error.html')

In [ ]:
fig = vis.plot_nodal_variable(swmm_heads_pd.transpose().to_numpy(), norm_sim_in_window, 'Target Head ', colorscale = 'PuBu', ref_marker_size = 2.5)

# Scratchpad (old)

In [ ]:
zero_cases_simulations_path = yaml_data['zero_cases_simulations_path']
list_of_zero_cases_simulations  = os.listdir(zero_cases_simulations_path)

zero_cases_simulations = utils.extract_simulations_from_folders(zero_cases_simulations_path, inp_path, max_events = 2)


In [ ]:
gnn_model = GNNModel()

In [ ]:
sim_trial = zero_cases_simulations[0]

length = sim_trial.simulation_length
sim_in_window = sim_trial.get_all_windows(steps_ahead = length-2)[0]
norm_sim_in_window = normalizer.normalize_window(sim_in_window)

name_nodes = norm_sim_in_window.name_nodes

swmm_heads_pd       = utils.tensor_heads_to_normalized_pd(sim_in_window['y'],             normalizer, name_nodes)
predicted_heads_pd  = utils.tensor_heads_to_normalized_pd(gnn_model(norm_sim_in_window),  normalizer, name_nodes)

In [ ]:
target      = normalizer.unnormalize_heads(norm_sim_in_window.y)
predicted   = normalizer.unnormalize_heads(gnn_model(norm_sim_in_window).detach())

In [ ]:
pd.DataFrame(target[-1,:].numpy()).plot()
pd.DataFrame(predicted[-1,:].numpy()).plot()

In [ ]:
error = torch.abs(target - predicted)
fig = vis.plot_nodal_variable(error, norm_sim_in_window, 'error in head at time: ', colorscale = 'OrRd')

In [ ]:
norm_sim_in_window.name_nodes[0]

In [ ]:
# fig.write_html('error.html')

In [ ]:
fig = vis.plot_nodal_variable(target, norm_sim_in_window, 'Target Head ', colorscale = 'PuBu', ref_marker_size = 2.5)

# Scratchpad (new)

In [27]:
val_event =  validation_simulations[0]
event_in_windows = val_event.get_all_windows(steps_ahead=1, steps_behind = 1)
print(val_event.name_simulation)

real_80


In [28]:
norm_event_in_windows = [normalizer.normalize_window(window) for window in event_in_windows]

In [29]:
results = []
for i in norm_event_in_windows:
    results.append(gnn_model(i))

RuntimeError: Trying to create tensor with negative dimension -6: [318, -6]

In [31]:
prediction = torch.cat(results, axis = 1)
unnormalized_prediction = normalizer.unnormalize_heads(prediction)

NotImplementedError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors, or that you (the operator writer) forgot to register a fallback function.  Available functions are [CPU, QuantizedCPU, BackendSelect, Python, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradLazy, AutogradXPU, AutogradMLC, AutogradHPU, AutogradNestedTensor, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, AutocastCPU, Autocast, Batched, VmapMode, Functionalize].

CPU: registered at aten\src\ATen\RegisterCPU.cpp:21063 [kernel]
QuantizedCPU: registered at aten\src\ATen\RegisterQuantizedCPU.cpp:1258 [kernel]
BackendSelect: fallthrough registered at ..\aten\src\ATen\core\BackendSelectFallbackKernel.cpp:3 [backend fallback]
Python: registered at ..\at

In [30]:
val_event =  validation_simulations[17]

length = val_event.simulation_length
sim_in_window = val_event.get_all_windows(steps_ahead = length-2)[0]
norm_sim_in_window = normalizer.normalize_window(sim_in_window)

name_nodes = norm_sim_in_window.name_nodes

swmm_heads_pd       = utils.tensor_heads_to_normalized_pd(sim_in_window['y'],             normalizer, name_nodes)

IndexError: list index out of range

In [ ]:
name_nodes =norm_sim_in_window.name_nodes

In [ ]:
name_nodes.index('j_90507') #j_90450

In [ ]:
rollout_results = gnn_model(norm_sim_in_window)
unnormalized_rollout = normalizer.unnormalize_heads(rollout_results)

In [ ]:
node = np.random.randint(0, 300)

print('Node: ', node, 'ID: ', name_nodes[node])
pd.DataFrame(unnormalized_prediction[node,:].detach().numpy()).plot()
swmm_heads_pd.iloc[:,node].plot()


import matplotlib.pyplot as plt
a = pd.DataFrame(unnormalized_rollout[node,:].detach().numpy())
b = pd.DataFrame(swmm_heads_pd.iloc[:,node])

fig, ax = plt.subplots()

ax.plot(a)
ax.plot(b)

plt.show()

In [ ]:
val_event.name_simulation

In [ ]:
norm_sim_in_window

In [ ]:
win = normalized_training_windows[0]

In [ ]:
gnn_model = GNNModel()

In [ ]:
gnn_model(win)

In [ ]:
norm_event_in_windows[0]

In [21]:
validation_windows[0].name_nodes

['j_9006F',
 'j_90334',
 'j_90335',
 'j_90336',
 'j_90363',
 'j_90364',
 'j_90365',
 'j_90366',
 'j_90367',
 'j_90368',
 'j_90369',
 'j_90370',
 'j_90371',
 'j_90372',
 'j_90373',
 'j_90374',
 'j_90375',
 'j_90376',
 'j_90377',
 'j_90378',
 'j_90379',
 'j_90380',
 'j_90381',
 'j_90382',
 'j_90383',
 'j_90384',
 'j_90385',
 'j_90386',
 'j_90387',
 'j_90388',
 'j_90389',
 'j_90390',
 'j_90391',
 'j_90392',
 'j_90393',
 'j_90394',
 'j_90395',
 'j_90396',
 'j_90397',
 'j_90398',
 'j_90399',
 'j_90400',
 'j_90401',
 'j_90402',
 'j_90403',
 'j_90404',
 'j_90405',
 'j_90406',
 'j_90407',
 'j_90408',
 'j_90427',
 'j_90428',
 'j_90430',
 'j_90433',
 'j_90434',
 'j_90435',
 'j_90436',
 'j_90437',
 'j_90438',
 'j_90439',
 'j_90440',
 'j_90441',
 'j_90442',
 'j_90443',
 'j_90444',
 'j_90445',
 'j_90446',
 'j_90447',
 'j_90448',
 'j_90449',
 'j_90450',
 'j_90451',
 'j_90452',
 'j_90453',
 'j_90454',
 'j_90455',
 'j_90456',
 'j_90457',
 'j_90459',
 'j_90460',
 'j_90461',
 'j_90462',
 'j_90463',
 'j_

In [ ]:
pred = torch.zeros(20, 15)  

In [ ]:
steps_behind